In [6]:
import sys
sys.path.append('/thesis')

In [7]:
import json
import pickle
# import logging
# import time

import numpy as np
import pandas as pd
# import matplotlib.pyplot as plt

from helper.character_encoder import DictionaryCharacterEncoder
from helper.prediction import predict_sequence

from implementation.seq2seq.transformer.utils import masked_loss, masked_accuracy
from implementation.seq2seq.transformer.keras_nlp import prepare_batches
from implementation.seq2seq.transformer.keras_nlp import construct_model_w_teacher_forcing

import tensorflow as tf
import keras_nlp

tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [8]:
# Input file paths
# jrc_file = '/thesis/data/jrc_person_pairs.csv'
wikidata_file = '/thesis/data/wikidata_person_to_en_norm.csv'

model_serialization_path = '/thesis/models/transformer_wikidata_to_en_1'

## Loading & Preprocessing

In [9]:
# Preprocessing
MAX_SEQ_LENGTH = 30  # 40
NUM_SAMPLES = 350000
VALIDATION_SPLIT = 0.3
RANDOM_STATE = 1010
BATCH_SIZE = 64

# Model
EMBEDDING_DIM = 64
INTERMEDIATE_DIM = 512
NUM_ENCODER_HEADS = 8
NUM_DECODER_HEADS = 8
NUM_ENCODER_LAYERS = 3
NUM_DECODER_LAYERS = 6

DROPOUT = 0.2

# Training
EPOCHS = 60
CHECKPOINT_FREQ = 10

In [10]:
# pairs_df = pd.read_csv(jrc_file, sep='|', encoding='utf-8')[['input', 'target']]
# pairs_df = pairs_df[(pairs_df['input'].str.len() <= MAX_SEQ_LENGTH) & (pairs_df['target'].str.len() <= MAX_SEQ_LENGTH)]
# print('Number of JRC pairs:', len(pairs_df))
# pairs_df2 = pd.read_csv(wikidata_file, sep='|', encoding='utf-8')[['input', 'target']]
# pairs_df2 = pairs_df2[(pairs_df2['input'].str.len() <= MAX_SEQ_LENGTH) & (pairs_df2['target'].str.len() <= MAX_SEQ_LENGTH)]
# print('Number of Wikidata pairs:', len(pairs_df2), '\n')
# pairs_df = pd.concat([pairs_df, pairs_df2]).sample(frac=1, random_state=RANDOM_STATE)

pairs_df = pd.read_csv(wikidata_file, sep='|', encoding='utf-8')[['input', 'target']]
pairs_df = pairs_df[(pairs_df['input'].str.len() <= MAX_SEQ_LENGTH) & (pairs_df['target'].str.len() <= MAX_SEQ_LENGTH)]
pairs_df

,input,target
0,duglass adamss,douglas adams
1,diego velasquez,diego velazquez
2,djego velaskess,diego velazquez
3,diego velasquez,diego velazquez
4,augusto pinocets,augusto pinochet
...,...,...
377875,daniil dushevskiy,daniil duseuski
377876,daniil dushevskiy,daniil duseuski
377877,daniil dushevskiy,daniil duseuski
377878,daniil dushevskiy,daniil duseuski


In [11]:
dce = DictionaryCharacterEncoder(max_seq_length=MAX_SEQ_LENGTH+2)
train_batches, val_batches = prepare_batches(pairs_df, dce, NUM_SAMPLES, VALIDATION_SPLIT, BATCH_SIZE, RANDOM_STATE)

2023-05-27 09:21:31.929973: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-05-27 09:21:31.930228: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-05-27 09:21:31.930400: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-05-27 09:21:33.144360: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-05-27 09:21:33.144609: I tensorflow/compile

In [12]:
for inputs, targets in train_batches.take(1):
    print(f'Encoder inputs shape: {inputs[0].shape}')
    print(f'Decoder inputs shape: {inputs[1].shape}')
    print(f'Targets shape: {targets.shape}')

Encoder inputs shape: (64, 30)
Decoder inputs shape: (64, 31)
Targets shape: (64, 31)


2023-05-27 09:21:41.715004: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype variant and shape [245000]
	 [[{{node Placeholder/_0}}]]
2023-05-27 09:21:41.715219: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype variant and shape [245000]
	 [[{{node Placeholder/_1}}]]


## Model

In [13]:
transformer = construct_model_w_teacher_forcing(
    num_encoder_layers=NUM_ENCODER_LAYERS, 
    num_decoder_layers=NUM_DECODER_LAYERS, 
    unique_tokens=len(dce.charset),
    max_seq_length=MAX_SEQ_LENGTH,
    embedding_dim=EMBEDDING_DIM,
    intermediate_dim=INTERMEDIATE_DIM,
    encoder_heads=NUM_ENCODER_HEADS,
    decoder_heads=NUM_DECODER_HEADS,
    dropout=DROPOUT
)

In [14]:
transformer.summary()
# transformer.compile("rmsprop", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
transformer.compile(
    loss=masked_loss,
    optimizer=tf.keras.optimizers.Adam(),
    metrics=[masked_accuracy])

Model: "transformer_w_tf"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 encoder_inputs (InputLayer)    [(None, 32)]         0           []                               
                                                                                                  
 embedding (Embedding)          (None, 32, 64)       2176        ['encoder_inputs[0][0]']         
                                                                                                  
 position_embedding (PositionEm  (None, 32, 64)      2048        ['embedding[0][0]']              
 bedding)                                                                                         
                                                                                                  
 tf.__operators__.add (TFOpLamb  (None, 32, 64)      0           ['embedding[0][0]'

## Training

In [15]:
steps_per_epoch = len(train_batches)

checkpoint_path = f'{model_serialization_path}/checkpoints/' + 'weights-{epoch:03d}'
checkpoint = tf.keras.callbacks.ModelCheckpoint(
                checkpoint_path, 
                monitor='val_masked_accuracy', 
                save_weights_only=True,
                save_freq=int(steps_per_epoch * CHECKPOINT_FREQ), 
                verbose=1)

In [16]:
history = transformer.fit(train_batches, epochs=EPOCHS, validation_data=val_batches, callbacks=[checkpoint])

Epoch 1/60


2023-05-27 09:21:43.981359: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype variant and shape [245000]
	 [[{{node Placeholder/_0}}]]
2023-05-27 09:21:43.981565: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype variant and shape [245000]
	 [[{{node Placeholder/_1}}]]
2023-05-27 09:21:53.779293: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:637] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-05-27 09:21:54.233819: I tensorflow/compiler/xla/service/service.cc:169] XLA service 0x67f7350 initialized for platfor

3829/3829 [==============================] - ETA: 0s - loss: 0.8318 - masked_accuracy: 0.7830

2023-05-27 09:25:06.108870: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype variant and shape [105000]
	 [[{{node Placeholder/_0}}]]
2023-05-27 09:25:06.109150: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype variant and shape [105000]
	 [[{{node Placeholder/_1}}]]


3829/3829 [==============================] - 236s 56ms/step - loss: 0.8318 - masked_accuracy: 0.7830 - val_loss: 0.4390 - val_masked_accuracy: 0.8899
Epoch 2/60
3829/3829 [==============================] - 220s 58ms/step - loss: 0.4598 - masked_accuracy: 0.8842 - val_loss: 0.3795 - val_masked_accuracy: 0.9041
Epoch 3/60
3829/3829 [==============================] - 213s 56ms/step - loss: 0.4088 - masked_accuracy: 0.8955 - val_loss: 0.3517 - val_masked_accuracy: 0.9100
Epoch 4/60
3829/3829 [==============================] - 213s 56ms/step - loss: 0.3810 - masked_accuracy: 0.9017 - val_loss: 0.3331 - val_masked_accuracy: 0.9143
Epoch 5/60
3829/3829 [==============================] - 215s 56ms/step - loss: 0.3633 - masked_accuracy: 0.9056 - val_loss: 0.3173 - val_masked_accuracy: 0.9169
Epoch 6/60
3829/3829 [==============================] - 214s 56ms/step - loss: 0.3506 - masked_accuracy: 0.9083 - val_loss: 0.3068 - val_masked_accuracy: 0.9188
Epoch 7/60
3829/3829 [=======================

In [17]:
transformer.save(f'{model_serialization_path}/')

2023-05-27 12:54:42.466387: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,?,64]
	 [[{{node inputs}}]]
2023-05-27 12:54:42.476337: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'decoder_sequence' with dtype float and shape [?,?,64]
	 [[{{node decoder_sequence}}]]
2023-05-27 12:54:42.485923: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'decoder_sequence' with dtype float and shape [?,?,64]
	 [[{

INFO:tensorflow:Assets written to: /thesis/models/transformer_wikidata_to_en_1/assets


INFO:tensorflow:Assets written to: /thesis/models/transformer_wikidata_to_en_1/assets


In [18]:
with open(model_serialization_path + '/train_history.p', 'wb') as file_pi:
    pickle.dump(history.history, file_pi)

train_config = {
    'batch_size': BATCH_SIZE,
    'epochs': EPOCHS,
    'num_samples': NUM_SAMPLES,
    'max_seq_length': MAX_SEQ_LENGTH,
    'random_state': RANDOM_STATE,
    'validation_split': VALIDATION_SPLIT,
    'encoder_layers': NUM_ENCODER_LAYERS,
    'decoder_layers': NUM_DECODER_LAYERS,
    'encoder_heads': NUM_ENCODER_HEADS,
    'decoder_heads': NUM_DECODER_HEADS,
    'dropout': DROPOUT,
    'embedding_dim': EMBEDDING_DIM,
    'intermediate_dim': INTERMEDIATE_DIM
}

with open(model_serialization_path + '/config.p', 'wb') as file_pi:
    pickle.dump(train_config, file_pi)

In [19]:
transformer = tf.keras.models.load_model(f'{model_serialization_path}/', custom_objects={'masked_loss': masked_loss, 'masked_accuracy': masked_accuracy})

## Prediction

In [20]:
def decode_sequences(input_sentence):
     # Tokenize the encoder input.
    encoder_input_tokens = tf.keras.utils.pad_sequences(
        dce.to_ids([input_sentence], insert_markers=True), 
        padding='post', 
        maxlen=MAX_SEQ_LENGTH+2
    )

    # Define a function that outputs the next token's probability given the
    # input sequence.
    def token_probability_fn(decoder_input_tokens):
        return transformer([encoder_input_tokens, decoder_input_tokens])[:, -1, :]


    prompt = tf.fill((1, 1), dce.char_index['\t'])
    generated_tokens = keras_nlp.utils.top_p_search(
        token_probability_fn,
        prompt,
        p=0.1,
        max_length=MAX_SEQ_LENGTH,
        end_token_id=dce.char_index['\n'],
    )
    generated_sentences = ''.join([dce.inverse_char_index[tkn] for tkn in generated_tokens.numpy()[0]])
    return generated_sentences.strip()

names = [
    'samuel meyer',
    'dmitry medvedev',
    'paulo ricardo',
    'zouheir al qaissi',
    'tarek al bichri',
    'thorsten brotzmann'
]

for s in names:
    translated = decode_sequences(s)
    print(s)
    print(translated)
    print()


samuel meyer
sam meyer

dmitry medvedev
dmitri medvedev

paulo ricardo
paul ricardo

zouheir al qaissi
zuheir al-qaissi

tarek al bichri
tarek al-bichri

thorsten brotzmann
thorsten broetzmann

